In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# !pip install imutils

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
# import system libs
import os
import time
import shutil
import pathlib
import itertools
from PIL import Image

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers
# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print ('modules loaded')

In [ ]:
folder_name = 'cleaned'
colab_path='/kaggle/working/'
colab_folder_path = f'{colab_path}/{folder_name}'
if not os.path.exists(colab_folder_path):
    os.makedirs(colab_folder_path)
    print(f'Folder "{folder_name}" created.')
    
folder_name = 'Training'
colab_path='/kaggle/working/cleaned/'
colab_folder_path = f'{colab_path}/{folder_name}'
if not os.path.exists(colab_folder_path):
    os.makedirs(colab_folder_path)
    print(f'Folder "{folder_name}" created.')

folder_name = 'Testing'
colab_path='/kaggle/working/cleaned/'
colab_folder_path = f'{colab_path}/{folder_name}'
if not os.path.exists(colab_folder_path):
    os.makedirs(colab_folder_path)
    print(f'Folder "{folder_name}" created.')


In [ ]:
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import numpy as np 
from tqdm import tqdm
import cv2
import os
import imutils

In [ ]:
import os
i=0
IMG_SIZE = 256
save_path = '/kaggle/working/cleaned/Training'
def print_files_in_folders(root_dir,save_path,i):
    for folder_name, subfolders, filenames in os.walk(root_dir):
        if 'test_images' in folder_name:
            continue  # Skip the 'test_images' folder
#         print(f"Folder: {folder_name}")
        for filename in filenames:
            file_path = os.path.join(folder_name, filename)
#             print(f"File path: {file_path}")
            image = cv2.imread(file_path)
            new_img = cv2.resize(image,(IMG_SIZE,IMG_SIZE))
            cv2.imwrite(save_path+'/'+filename, new_img)
        i+=1
    print(i)

# Replace 'path_to_your_folder' with the path to the folder you want to start from
folder_path = '/kaggle/input/eeg-5-20-60-40/5_20_60_40/Train'
print_files_in_folders(folder_path,save_path,i)

In [ ]:
import os
i=0
IMG_SIZE = 256
save_path = '/kaggle/working/cleaned/Testing'
def print_files_in_folders(root_dir,save_path,i):
    for folder_name, subfolders, filenames in os.walk(root_dir):
        if 'test_images' in folder_name:
            continue  # Skip the 'test_images' folder
#         print(f"Folder: {folder_name}")
        for filename in filenames:
            file_path = os.path.join(folder_name, filename)
#             print(f"File path: {file_path}")
            image = cv2.imread(file_path)
            new_img = cv2.resize(image,(IMG_SIZE,IMG_SIZE))
            cv2.imwrite(save_path+'/'+filename, new_img)
        i+=1
    print(i)

# Replace 'path_to_your_folder' with the path to the folder you want to start from
folder_path = '/kaggle/input/eeg-5-20-60-40/5_20_60_40/Test'
print_files_in_folders(folder_path,save_path,i)

In [ ]:
import os

folder_path = '/kaggle/working/cleaned/Testing'
files_count = sum(len(files) for _, _, files in os.walk(folder_path))
print(files_count)

In [ ]:
import os

folder_path = '/kaggle/working/cleaned/Training'
files_count = sum(len(files) for _, _, files in os.walk(folder_path))
print(files_count)

In [ ]:
import os       
# Generate data paths with labels
train_data_dir = '/kaggle/working/cleaned/Training'
filepaths = []
labels = []

for root, dirs, files in os.walk(train_data_dir):
    for file in files:
        fpath=os.path.join(root, file)
        filepaths.append(fpath)
        if 'nor' in file :
            labels.append('normal')
        else:
            labels.append('abnormal')
        
        
# Concatenate data paths with labels into one dataframe
Fseries = pd.Series(filepaths, name= 'filepaths')
Lseries = pd.Series(labels, name='labels')
train_df = pd.concat([Fseries, Lseries], axis= 1)

In [ ]:
train_df

In [ ]:
import os       
# Generate data paths with labels
train_data_dir = '/kaggle/working/cleaned/Testing'
filepaths = []
labels = []

for root, dirs, files in os.walk(train_data_dir):
    for file in files:
        fpath=os.path.join(root, file)
        filepaths.append(fpath)
        if 'nor' in file :
            labels.append('normal')
        else:
            labels.append('abnormal')
        
        
# Concatenate data paths with labels into one dataframe
Fseries = pd.Series(filepaths, name= 'filepaths')
Lseries = pd.Series(labels, name='labels')
ts_df = pd.concat([Fseries, Lseries], axis= 1)

In [ ]:
ts_df

In [ ]:
train_df, valid_df = train_test_split(train_df,  train_size= 0.8, shuffle= True, random_state= 123)

In [ ]:
batch_size = 16
img_size = (256, 256)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

valid_gen = tr_gen.flow_from_dataframe( valid_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

In [ ]:
g_dict = train_gen.class_indices      # defines dictionary {'class': index}
classes = list(g_dict.keys())       # defines list of dictionary's kays (classes), classes names : string
images, labels = next(train_gen)      # get a batch size samples from the generator

plt.figure(figsize= (20, 20))

for i in range(16):
    plt.subplot(4, 4, i + 1)
    image = images[i] / 255       # scales data to range (0 - 255)
    plt.imshow(image)
    index = np.argmax(labels[i])  # get image index
    class_name = classes[index]   # get class of image
    plt.title(class_name, color= 'blue', fontsize= 12)
    plt.axis('off')
plt.show()

In [ ]:
img_size = (256, 256)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_gen.class_indices.keys())) # to define number of classes in dense layer

epochs = 30  

In [ ]:
class_indices = train_gen.class_indices

# Reverse the mapping to get class names from indices
class_names = {v: k for k, v in class_indices.items()}

print("Class Names:", class_names)

In [ ]:
base_model = tf.keras.applications.EfficientNetB3(include_top= False, weights= "imagenet", input_shape= img_shape, pooling= 'max')

model = Sequential([
    base_model,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, kernel_regularizer= regularizers.l2(l= 0.016), activity_regularizer= regularizers.l1(0.006),
                bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
    Dropout(rate= 0.45, seed= 123),
    Dense(class_count, activation= 'softmax')
])

model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

In [ ]:
history = model.fit(x= train_gen, epochs= epochs, verbose= 1, validation_data= valid_gen, 
                    validation_steps= None, shuffle= False)

In [ ]:
# Define needed variables
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]
index_acc = np.argmax(val_acc)
acc_highest = val_acc[index_acc]
Epochs = [i+1 for i in range(len(tr_acc))]
loss_label = f'best epoch= {str(index_loss + 1)}'
acc_label = f'best epoch= {str(index_acc + 1)}'

# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.subplot(1, 2, 1)
plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

In [ ]:
# model.save_weights("/kaggle/working/EfficientNetB3_5_20_normal_threshold_softmax_60_40_latest.h5")

In [ ]:

# model.load_weights('/kaggle/input/weight-efficient/EfficientNetB3_5_20_normal_threshold_softmax_30_epoch.h5')  # Replace with the actual path to your weights file

In [ ]:
file_paths = ts_df['filepaths'].tolist()

In [ ]:
ts_df

In [ ]:
class_indices = train_gen.class_indices

# Reverse the mapping to get class names from indices
class_names = {v: k for k, v in class_indices.items()}

print("Class Names:", class_names)

In [ ]:
file_paths = ts_df['filepaths'].tolist()

In [ ]:
pred_labels=[]
for file_path in file_paths:
    img = cv2.imread(file_path)
    single_pred = model.predict(np.expand_dims(img, axis=0))
    y_pred = np.argmax(single_pred, axis=1)
    pred_labels.append(class_names[y_pred[0]])
    

In [ ]:
ts_df['pred_class']=pred_labels
ts_df

In [ ]:
import os

def list_folders_in_path(path):
    folders_list = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]
    return folders_list

# Replace 'your/path/here' with the path you want to explore
path = '/kaggle/input/eeg-5-20-60-40/5_20_60_40/Test'
folders = list_folders_in_path(path)
print("Folders in the path:", folders)


In [ ]:
channel_names=['F7','F3','F4','F8','T3','C3','Cz','C4','T4','T5','P3','Pz','P4','T6','O1','O2']

In [ ]:
mapping = {'normal':0, 'abnormal':1}

# Mapping values using dictionary comprehension
ts_df['labels'] = ts_df['labels'].map(mapping)

In [ ]:
ts_df['pred_class'] = ts_df['pred_class'].map(mapping)

In [ ]:
ts_df

In [ ]:
# import pandas as pd
# from sklearn.metrics import accuracy_score



# # Calculate accuracy
# accuracy = accuracy_score(ts_df['labels'], ts_df['pred_class'])

# print(f"Accuracy: {accuracy:.2f}")

In [ ]:
channel_names=['F7','F3','F4','F8','T3','C3','Cz','C4','T4','T5','P3','Pz','P4','T6','O1','O2']
for names in folders:
    x=0
    print(names)
    channel_counts = {channel: 0 for channel in channel_names}
    folder_count=0
    for index, row in ts_df.iterrows():
#         print(row)
        if names in row[0]:
            for channel in channel_names:
                if channel in row[0]:
                    x+=1
                    channel_counts[channel]+=row[2]
                    folder_count+=row[2]
#                 print(x)
#                 x=0
#     break
    print(names,channel_counts,folder_count)

In [ ]:
channel_names=['F7','F3','F4','F8','T3','C3','Cz','C4','T4','T5','P3','Pz','P4','T6','O1','O2']
x=[]
for names in folders:
#     x=0
    print(names)
    channel_counts = {channel: 0 for channel in channel_names}
    folder_count=0
    for index, row in ts_df.iterrows():
#         print(row)
        if names in row[0]:
            for channel in channel_names:
                if channel in row[0]:
#                     x+=1
                    channel_counts[channel]+=row[2]
                    folder_count+=row[2]
#                 print(x)
#                 x=0
#     break
    print([names,channel_counts,folder_count])
    x.append([names,channel_counts,folder_count])

In [ ]:
val_df=pd.DataFrame(x,columns=['file_names','dict','count'])
val_df

In [ ]:
df_expanded = pd.json_normalize(val_df['dict'])

# Concatenate the expanded columns with the original DataFrame
result = pd.concat([val_df.drop('dict', axis=1), df_expanded], axis=1)
result

In [ ]:
# channel_names

# Checking values and converting them to 1 if greater than half
for column in channel_names:
    result[column] = result[column].apply(lambda x: 1 if x >= 4 else 0)

# Display the updated DataFrame
# print(result)
result

In [ ]:
result['channel_count_sum'] = result[channel_names].sum(axis=1)

In [ ]:
result

In [ ]:
result['channel_count_sum'] = result['channel_count_sum'].apply(lambda x: 1 if x >= 8 else 0)
result

In [ ]:
result['channel_count_wise'] = result['count'].apply(lambda x: 1 if x >= 72 else 0)
result

In [ ]:
result['ground_truth']=0

In [ ]:
result.info()

In [ ]:
for index, row in result.iterrows():
#     print(row[0])
    if 'nor' in row['file_names']:
        result.at[index, 'ground_truth'] = 0
    else:
        result.at[index, 'ground_truth'] = 1
result
        

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score



# Calculate accuracy
accuracy = accuracy_score(result['ground_truth'], result['channel_count_wise'])

print(f"Accuracy: {accuracy:.2f}")

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score



# Calculate accuracy
accuracy = accuracy_score(result['ground_truth'], result['channel_count_sum'])

print(f"Accuracy: {accuracy:.2f}")

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score



# Calculate accuracy
accuracy = accuracy_score(result['ground_truth'], result['pred_class'])

print(f"Accuracy: {accuracy:.2f}")

In [ ]:
folder_name = 'single_channel_combined_Testing'
colab_path='/kaggle/working/cleaned/'
colab_folder_path = f'{colab_path}/{folder_name}'
if not os.path.exists(colab_folder_path):
    os.makedirs(colab_folder_path)
    print(f'Folder "{folder_name}" created.')


In [ ]:
import os
i=0
IMG_SIZE = 256
save_path = '/kaggle/working/cleaned/single_channel_combined_Testing'
def print_files_in_folders(root_dir,save_path,i):
    for folder_name, subfolders, filenames in os.walk(root_dir):
        if 'test_images' in folder_name:
            continue  # Skip the 'test_images' folder
#         print(f"Folder: {folder_name}")
        for filename in filenames:
            file_path = os.path.join(folder_name, filename)
#             print(f"File path: {file_path}")
            image = cv2.imread(file_path)
            new_img = cv2.resize(image,(IMG_SIZE,IMG_SIZE))
            cv2.imwrite(save_path+'/'+filename, new_img)
        i+=1
    print(i)

# Replace 'path_to_your_folder' with the path to the folder you want to start from
folder_path = '/kaggle/input/eeg-tes/all_images_scalogram_combined_single_channel_final/content/all_images_scalogram_combined_single_channel_final'
print_files_in_folders(folder_path,save_path,i)


In [ ]:
import os

folder_path = '/kaggle/working/cleaned/single_channel_combined_Testing'
files_count = sum(len(files) for _, _, files in os.walk(folder_path))
print(files_count)


In [ ]:

import os       
# Generate data paths with labels
train_data_dir = '/kaggle/working/cleaned/single_channel_combined_Testing'
filepaths = []
labels = []

for root, dirs, files in os.walk(train_data_dir):
    for file in files:
        fpath=os.path.join(root, file)
        filepaths.append(fpath)
        if 'nor' in file :
            labels.append('normal')
        else:
            labels.append('abnormal')

Fseries = pd.Series(filepaths, name= 'filepaths')
Lseries = pd.Series(labels, name='labels')
ts_df = pd.concat([Fseries, Lseries], axis= 1)

In [ ]:
file_paths = ts_df['filepaths'].tolist()

pred_labels=[]
for file_path in file_paths:
    img = cv2.imread(file_path)
    single_pred = model.predict(np.expand_dims(img, axis=0))
    y_pred = np.argmax(single_pred, axis=1)
    pred_labels.append(class_names[y_pred[0]])


In [ ]:
ts_df['pred_class']=pred_labels

ts_df['pred_class'] = ts_df['pred_class'].map(mapping)
ts_df

In [ ]:
ts_df

In [ ]:
channel_names=['F7','F3','F4','F8','T3','C3','Cz','C4','T4','T5','P3','Pz','P4','T6','O1','O2']
x=[]
for names in folders:
#     x=0
    print(names)
    channel_counts = {channel: 0 for channel in channel_names}
    folder_count=0
    for index, row in ts_df.iterrows():
#         print(row)
        if names in row[0]:
#             for channel in channel_names:
#                 if channel in row[0]:
#                     x+=1
#                     channel_counts[channel]+=row[2]
                    folder_count+=row[2]
#                 print(x)
#                 x=0
#     break
    print([names,folder_count])
    x.append([names,folder_count])
    



In [ ]:
val_df=pd.DataFrame(x,columns=['file_names','count'])
val_df



In [ ]:
val_df['ground_truth']=0

In [ ]:
for index, row in val_df.iterrows():
#     print(row[0])
    if 'nor' in row['file_names']:
        val_df.at[index, 'ground_truth'] = 0
    else:
        val_df.at[index, 'ground_truth'] = 1
val_df
        

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score



# Calculate accuracy
accuracy = accuracy_score(val_df['ground_truth'], val_df['count'])

print(f"Accuracy: {accuracy:.2f}")

In [ ]:
folder_name = 'channel_wise_Testing'
colab_path='/kaggle/working/cleaned/'
colab_folder_path = f'{colab_path}/{folder_name}'
if not os.path.exists(colab_folder_path):
    os.makedirs(colab_folder_path)
    print(f'Folder "{folder_name}" created.')


import os
i=0
IMG_SIZE = 256
save_path = '/kaggle/working/cleaned/channel_wise_Testing'
def print_files_in_folders(root_dir,save_path,i):
    for folder_name, subfolders, filenames in os.walk(root_dir):
        if 'test_images' in folder_name:
            continue  # Skip the 'test_images' folder
#         print(f"Folder: {folder_name}")
        for filename in filenames:
            file_path = os.path.join(folder_name, filename)
#             print(f"File path: {file_path}")
            image = cv2.imread(file_path)
            new_img = cv2.resize(image,(IMG_SIZE,IMG_SIZE))
            cv2.imwrite(save_path+'/'+filename, new_img)
        i+=1
    print(i)

# Replace 'path_to_your_folder' with the path to the folder you want to start from
folder_path = '/kaggle/input/eeg-tes/all_images_scalogram/content/all_images_scalogram'
print_files_in_folders(folder_path,save_path,i)

import os

folder_path = '/kaggle/working/cleaned/channel_wise_Testing'
files_count = sum(len(files) for _, _, files in os.walk(folder_path))
print(files_count)



In [ ]:
import os       
# Generate data paths with labels
train_data_dir = '/kaggle/working/cleaned/channel_wise_Testing'
filepaths = []
labels = []

for root, dirs, files in os.walk(train_data_dir):
    for file in files:
        fpath=os.path.join(root, file)
        filepaths.append(fpath)
        if 'nor' in file :
            labels.append('normal')
        else:
            labels.append('abnormal')
        
        
# Concatenate data paths with labels into one dataframe
Fseries = pd.Series(filepaths, name= 'filepaths')
Lseries = pd.Series(labels, name='labels')
ts_df = pd.concat([Fseries, Lseries], axis= 1)

file_paths = ts_df['filepaths'].tolist()

pred_labels=[]
for file_path in file_paths:
    img = cv2.imread(file_path)
    single_pred = model.predict(np.expand_dims(img, axis=0))
    y_pred = np.argmax(single_pred, axis=1)
    pred_labels.append(class_names[y_pred[0]])

    
ts_df['pred_class']=pred_labels


ts_df['pred_class'] = ts_df['pred_class'].map(mapping)
ts_df

In [ ]:
channel_names=['F7','F3','F4','F8','T3','C3','Cz','C4','T4','T5','P3','Pz','P4','T6','O1','O2']
x=[]
for names in folders:
#     x=0
    print(names)
    channel_counts = {channel: 0 for channel in channel_names}
    folder_count=0
    for index, row in ts_df.iterrows():
#         print(row)
        if names in row[0]:
            for channel in channel_names:
                if channel in row[0]:
#                     x+=1
                    channel_counts[channel]+=row[2]
                    folder_count+=row[2]
#                 print(x)
#                 x=0
#     break
    print([names,channel_counts,folder_count])
    x.append([names,channel_counts,folder_count])
    



In [ ]:
val_df=pd.DataFrame(x,columns=['file_names','dict','count'])
# val_df

df_expanded = pd.json_normalize(val_df['dict'])

# Concatenate the expanded columns with the original DataFrame
result = pd.concat([val_df.drop('dict', axis=1), df_expanded], axis=1)
result



# channel_names


In [ ]:
result['ground_truth']=0

for index, row in result.iterrows():
#     print(row[0])
    if 'nor' in row['file_names']:
        result.at[index, 'ground_truth'] = 0
    else:
        result.at[index, 'ground_truth'] = 1
result
        

In [ ]:
result['pred_labels'] = result['count'].apply(lambda x: 1 if x >= 4 else 0)
result


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score



# Calculate accuracy
accuracy = accuracy_score(result['ground_truth'], result['pred_labels'])

print(f"Accuracy: {accuracy:.2f}")